In [ ]:
# April 2025
# Tags

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.chains import create_tagging_chain, create_tagging_chain_pydantic

In [ ]:
from dotenv import load_dotenv
import os

# 1st method: using .env file.
load_dotenv()
# Access them using os.getenv or os.environ
api_key = os.getenv("GROQ_API_KEY")

# 2nd method: using hard code
# api_key = "<put the api key here>"
# if not os.environ.get("GROQ_API_KEY"):
#     os.environ["GROQ_API_KEY"] = api_key #getpass.getpass("Enter API key for Groq: ")



from langchain_groq import ChatGroq

llm = ChatGroq(model="llama3-8b-8192")

In [ ]:
schema = {
    "properties": {
        # Schema definitions here
    },
    "required": ["language", "sentiment", "aggressiveness", "goal", "flow", "context"],
}

chain = create_tagging_chain(schema, llm)

In [ ]:
# inp = "Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!"
# chain.run(inp)
# {'sentiment': 'positive', 'language': 'Spanish'}

inp = "Estoy muy enojado con vos! Te voy a dar tu merecido!"
inp = """
Speaker A: Hey, how was your weekend?
Speaker B: It was, like, really chill, you know? Just hung out with friends and stuff.
Speaker A: That sounds awesome, man. I wish I had more free time to do that.
Speaker B: Yeah, I know, right? But, um, I did get some work done, too. Needed to catch up on some projects.
Speaker A: Yeah, I've been putting off some tasks, too. Guess we're both just trying to stay on top of things, huh?
Speaker B: Exactly! But, hey, did you hear about the new coffee shop that just opened up downtown?
Speaker A: Yeah, I saw that on social media. I've been meaning to check it out. Have you tried it?
Speaker B: Yeah, I went there yesterday. The coffee is really good, and the vibe is super cozy.
Speaker A: Nice! I'll have to go there this week. Thanks for the rec!
Speaker B: No problem, happy to help!
Speaker A: So, what do you think about the new traffic lights they installed on Main St?
Speaker B: Honestly, I think they're a total nightmare. It's like, now there's just more traffic and congestion.
Speaker A: Yeah, I agree. I was stuck in traffic for like, 20 minutes yesterday.
Speaker B: Ugh, I know. And it's not like it's even made a big difference in traffic flow or anything.
Speaker A: Yeah, it just feels like a band-aid solution or something.
Speaker B: Exactly. Anyway, have you seen that new movie that just came out?
Speaker A: Yeah, I saw it last night. It was really good! I loved the special effects.
Speaker B: Yeah, I've been wanting to see it. Did you think it was worth the hype?
Speaker A: Totally. I mean, the plot was a little confusing at times, but the visuals were amazing.
Speaker B: Yeah, I've heard that. I might have to wait till it comes out on streaming or something.
Speaker A: Yeah, that's probably a good idea. I don't know if I'd want to pay full price for it.
Speaker B: No way. I mean, there are just so many other good movies out right now.
Speaker A: For sure. But hey, have you tried that new restaurant that just opened up on the waterfront?
Speaker B: Yeah, I went there for dinner last week. The food was really good, but the service was kind of slow.
Speaker A: Yeah, I've heard that. I might have to try it out anyway. Thanks for the heads up!
Speaker B: No problem, happy to help!
"""
chain.run(inp)

In [ ]:
# Another way:

In [ ]:
from langchain.chains import create_tagging_chain

schema = {
    "properties": {
        "emotion": {
            "type": "string",
            "enum": ["angry", "sad", "happy", "neutral"],
            "description": "The dominant emotion expressed in the text."
        },
        "intent": {
            "type": "string",
            "description": "The underlying intent (e.g., request, complaint, compliment, threat)."
        },
        "clarity_level": {
            "type": "string",
            "enum": ["clear", "somewhat unclear", "very unclear",  "not clear"],
            "description": "How easy it is to understand what the speaker is saying."
        },
        "manipulative": {
            "type": "boolean",
            "description": "Whether the speaker is using emotionally or logically manipulative tactics."
        },
        "level_of_politeness": {
            "type": "string",
            "enum": ["very polite", "polite", "neutral", "impolite", "very impolite"]
        }
    },
    "required": ["emotion", "intent", "clarity_level", "manipulative",  "level_of_politeness"]
}

tagging_chain = create_tagging_chain(schema, llm)

input_text = "I absolutely love the new iPhone!"
result = tagging_chain.run(input_text)
print(result)


In [ ]:
from langchain.chains import create_tagging_chain_pydantic
from pydantic import BaseModel, Field

class TagOutput(BaseModel):
    sentiment: str = Field(description="Sentiment of the text: positive, neutral, or negative")
    topic: str = Field(description="Main topic discussed")

tagging_chain = create_tagging_chain_pydantic(TagOutput, llm)

result = tagging_chain.run("The economy is collapsing and it's terrifying.")
print(result)

In [ ]:
result.sentiment, result.topic

In [ ]:
# The following is an extention of the above code:

In [ ]:
# however, it's errorsome. and you need to change it to something like bellow:

In [ ]:
from langchain.chains import create_tagging_chain_pydantic
from pydantic import BaseModel, Field
from typing import Literal


class TagOutput(BaseModel):
    emotion: Literal["angry", "sad", "happy", "neutral"] = Field(
        description="The dominant emotion expressed in the text."
    )
    intent: str = Field(
        description="The underlying intent (e.g., request, complaint, compliment, threat)."
    )
    clarity_level: Literal["clear", "somewhat unclear", "very unclear", "not clear"] = Field(
        description="How easy it is to understand what the speaker is saying."
    )
    manipulative: Literal["true", "false"] = Field(
        description="Whether the speaker is using emotionally or logically manipulative tactics."
    )
    level_of_politeness: Literal["very polite", "polite", "neutral", "impolite", "very impolite"] = Field(
        description="Level of politeness of the speaker."
    )

tagging_chain = create_tagging_chain_pydantic(TagOutput, llm)

result = tagging_chain.run("The economy is collapsing and it's terrifying.")
print(result)    

In [ ]:
# Don't forget to use Literal as it's important to return bools etc. 
#  the underlying model (Groq's Mixtral here) returned it as a string ("false") instead of a boolean false.
# This type mismatch causes LangChain's validation to fail, resulting in a BadRequestError.

In [ ]:
result.schema().keys()

In [ ]:
from langchain.chains import create_tagging_chain_pydantic
from pydantic import BaseModel, Field
from typing import Literal


from typing import Literal, Optional
from pydantic import BaseModel, Field



from pydantic import BaseModel
from typing import Literal

class TagOutput(BaseModel):
    # 🌍 Basic Metadata
    language: str
    context: str

    # 💬 Linguistic/Discourse
    formality: Literal["very informal", "informal", "neutral", "formal", "very formal"]
    complexity: Literal["very simple", "simple", "moderate", "complex", "very complex"]
    coherence: Literal["incoherent", "somewhat coherent", "coherent", "very coherent"]
    verbosity: Literal["terse", "brief", "balanced", "verbose", "very verbose"]
    fluency: Literal["disfluent", "somewhat fluent", "fluent", "very fluent"]
    clarity: Literal["unclear", "somewhat clear", "clear", "very clear"]
    repetition: Literal["none", "low", "moderate", "high", "very high"]
    disfluency: Literal["none", "low", "moderate", "high", "very high"]

    # 😊 Emotional/Affective
    sentiment: Literal["very negative", "negative", "neutral", "positive", "very positive"]
    politeness: Literal["very impolite", "impolite", "neutral", "polite", "very polite"]
    empathy: Literal["none", "low", "moderate", "high", "very high"]
    aggressiveness: Literal["none", "low", "moderate", "high", "very high"]
    sarcasm: Literal["none", "low", "moderate", "high", "very high"]
    humor: Literal["none", "low", "moderate", "high", "very high"]
    hostility: Literal["none", "low", "moderate", "high", "very high"]
    emotional_intensity: Literal["none", "low", "moderate", "high", "very high"]



tagging_chain = create_tagging_chain_pydantic(TagOutput, llm)

result = tagging_chain.run("The economy is collapsing and it's terrifying.")
print(result)    

In [ ]:
result

In [ ]:
 from langchain.chains import create_tagging_chain_pydantic
from pydantic import BaseModel, Field
from typing import Literal


from typing import Literal, Optional
from pydantic import BaseModel, Field


class TagOutput(BaseModel):
    # 🧠 Cognitive/Reasoning
    confidence: Literal["none", "low", "moderate", "high", "very high"]
    certainty: Literal["very uncertain", "uncertain", "neutral", "certain", "very certain"]
    curiosity: Literal["none", "low", "moderate", "high", "very high"]
    confusion: Literal["none", "low", "moderate", "high", "very high"]
    deception: Literal["none", "low", "moderate", "high", "very high"]
    rationality: Literal["emotional", "mostly emotional", "balanced", "mostly logical", "logical"]

    # 🎯 Intent/Pragmatics
    goal: str
    intent_type: Literal["question", "inform", "persuade", "command", "express emotion", "other"]
    persuasiveness: Literal["none", "low", "moderate", "high", "very high"]
    manipulation: Literal["none", "low", "moderate", "high", "very high"]
    cooperation_level: Literal["none", "low", "moderate", "high", "very high"]
    topic_shift: Literal["none", "low", "moderate", "high"]
    reference_resolution: Literal["poor", "fair", "good", "very good"]

    # 🤝 Relational/Interactional
    dominance: Literal["submissive", "balanced", "dominant"]
    supportiveness: Literal["none", "low", "moderate", "high", "very high"]
    interruption_frequency: Literal["none", "low", "moderate", "high", "very high"]
    turn_taking_balance: Literal["speaker-heavy", "balanced", "listener-heavy"]
    alignment: Literal["disagree", "neutral", "agree"]
    relationship_type: Literal["peer", "superior", "subordinate", "customer", "service provider", "other"]

    # 📈 Meta/Structural
    flow: Literal["poor", "fair", "good", "very good"]
    engagement_level: Literal["none", "low", "moderate", "high", "very high"]
    topic_relevance: Literal["irrelevant", "somewhat relevant", "relevant", "very relevant"]
    new_information_ratio: Literal["none", "low", "moderate", "high", "very high"]
    self_disclosure_level: Literal["none", "low", "moderate", "high", "very high"]
    question_density: Literal["none", "low", "moderate", "high", "very high"]
    
tagging_chain = create_tagging_chain_pydantic(TagOutput, llm)

result = tagging_chain.run("The economy is collapsing and it's terrifying.")
print(result)        